<a href="https://colab.research.google.com/github/ByungjunKim/JKFR-100th-Question/blob/main/%ED%98%84%EB%8C%80%EC%86%8C%EC%84%A4%EC%97%B0%EA%B5%AC_%ED%8A%B9%EC%A7%91%EB%85%BC%EB%AC%B8(%EA%B8%B0%EC%B4%88_%ED%86%B5%EA%B3%84).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1. 한글 폰트 설치 (코랩 전용)
# ==========================================
# 실행 후 "런타임 다시 시작"이 필요할 수 있습니다. (폰트 깨짐 방지)
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

import matplotlib.pyplot as plt
plt.rc('font', family='NanumBarunGothic')
plt.rcParams['axes.unicode_minus'] = False # 마이너스 기호 깨짐 방지

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.font_manager as fm

# Matplotlib에서 한글 폰트 설정을 위한 임포트 및 마이너스 부호 처리
# 사용자 요청에 따라 'NanumBarunGothic' 폰트를 사용합니다. (설치 필수)
try:
    plt.rc('font', family='NanumBarunGothic')
    plt.rcParams['axes.unicode_minus'] = False
except Exception:
    # 폰트가 없을 경우 경고 메시지 출력. 그래프는 한글 깨짐 상태로 생성될 수 있습니다.
    print("경고: NanumBarunGothic 폰트를 찾을 수 없습니다. 기본 폰트로 대체됩니다.")

# 데이터 파일을 로드하고 연대를 구분하는 공통 로직을 수행하는 함수
def load_and_preprocess_data():
    """데이터를 로드하고 'year'를 기반으로 'Decade'를 생성하여 반환합니다."""
    try:
        # 데이터 로드
        file_path = '/content/[현대소설연구 번역서, 원서 참고문헌 최종] (논문 발행연도 포함).csv'
        df = pd.read_csv(file_path)
    except FileNotFoundError:
        print(f"오류: 파일을 찾을 수 없습니다. 경로를 확인해주세요: {file_path}")
        return None
    except Exception as e:
        print(f"데이터 로드 중 오류 발생: {e}")
        return None

    # 'year' 열을 정수형으로 변환 (오류 발생 시 해당 행은 제거)
    df['year'] = pd.to_numeric(df['year'], errors='coerce').astype('Int64')
    df.dropna(subset=['year'], inplace=True) # year 기준 null 제거

    # 1. 연대(Decade) 정의 함수
    def get_decade(year):
        """논문 발행 연도를 바탕으로 연대 구분"""
        if 1990 <= year <= 1999:
            return '1990년대 (1994-1999)'
        elif 2000 <= year <= 2009:
            return '2000년대 (2000-2009)'
        elif 2010 <= year <= 2019:
            return '2010년대 (2010-2019)'
        elif 2020 <= year:
            return '2020년대 (2020-현재)'
        return '기타'

    df['Decade'] = df['year'].apply(get_decade)

    # 분석에서 제외할 '기타' 연대 필터링
    df_filtered = df[df['Decade'] != '기타'].copy()

    return df_filtered


def analyze_authors(df_filtered):
    """
    제어번호(작가)를 기준으로 연대별 최다 인용 작가를 분석하고 결과를 CSV로 저장합니다.
    """
    if df_filtered.empty or '제어번호' not in df_filtered.columns:
        print("작가 분석을 위한 데이터 또는 '제어번호' 열이 부족합니다.")
        return

    # 2. 제어번호(작가)와 작가 이름(author) 매핑 생성
    # 'author_정제' 열(영어명/원어명)을 사용합니다.
    author_map = df_filtered.dropna(subset=['제어번호', 'author_정제']).groupby('제어번호')['author_정제'].agg(
        lambda x: x.mode()[0] if not x.mode().empty else 'Unknown_Author'
    ).to_dict()

    # 3. 연대별 인용 횟수 집계
    citation_counts = df_filtered.groupby(['Decade', '제어번호']).size().reset_index(name='Citation_Count')

    # 4. 연대별 최다 인용 작가 10명 선정
    top_authors_per_decade = (
        citation_counts.groupby('Decade')
        .apply(lambda x: x.sort_values(by='Citation_Count', ascending=False).head(10))
        .reset_index(drop=True)
    )

    # 5. 작가 이름(원어) 매핑 적용
    top_authors_per_decade['Author_Name_EN'] = top_authors_per_decade['제어번호'].map(author_map)

    # 6. CSV 파일 생성 및 저장
    csv_output = top_authors_per_decade[['Decade', 'Author_Name_EN', '제어번호', 'Citation_Count']].rename(
        columns={'Decade': '분석_년대', 'Author_Name_EN': '작가_이름(원어)', '제어번호': '작가_제어번호', 'Citation_Count': '인용_횟수'}
    )

    csv_filename = 'top_cited_authors_by_decade.csv'
    csv_output.to_csv(csv_filename, index=False, encoding='utf-8-sig')
    print(f"[작가 분석] CSV 파일 저장 완료: {csv_filename}")

    # 7. 시각화 데이터 반환 (그래프 생성 함수에서 사용)
    return csv_output

def analyze_works(df_filtered):
    """
    title_정제(작품)를 기준으로 연대별 최다 인용 작품을 분석하고 결과를 CSV로 저장합니다.
    """
    if df_filtered.empty or 'title_정제' not in df_filtered.columns:
        print("작품 분석을 위한 데이터 또는 'title_정제' 열이 부족합니다.")
        return

    # 1. 'title_정제' 열을 인용 대상(Work ID)으로 사용
    # 'title_정제'가 비어있지 않은 행만 사용
    df_works = df_filtered.dropna(subset=['title_정제']).copy()

    # 2. 연대별 인용 횟수 집계
    work_citation_counts = df_works.groupby(['Decade', 'title_정제']).size().reset_index(name='Citation_Count')

    # 3. 연대별 최다 인용 작품 10명 선정
    top_works_per_decade = (
        work_citation_counts.groupby('Decade')
        .apply(lambda x: x.sort_values(by='Citation_Count', ascending=False).head(10))
        .reset_index(drop=True)
    )

    # 4. CSV 파일 생성 및 저장
    csv_output = top_works_per_decade[['Decade', 'title_정제', 'Citation_Count']].rename(
        columns={'Decade': '분석_년대', 'title_정제': '작품_이름(정제)', 'Citation_Count': '인용_횟수'}
    )

    csv_filename = 'top_cited_works_by_decade.csv'
    csv_output.to_csv(csv_filename, index=False, encoding='utf-8-sig')
    print(f"[작품 분석] CSV 파일 저장 완료: {csv_filename}")

    # 5. 시각화 데이터 반환 (그래프 생성 함수에서 사용)
    return csv_output

# =========================================================
# 그래프 시각화 함수 (공통)
# =========================================================

def create_and_save_plot(data, title_prefix, filename):
    """최다 인용 순위를 연대별로 시각화하여 PNG 파일로 저장"""

    if data is None or data.empty:
        print(f"시각화를 위한 데이터가 없습니다: {title_prefix}")
        return

    decades = data['분석_년대'].unique()

    fig, axes = plt.subplots(len(decades), 1, figsize=(12, 5 * len(decades)))
    fig.suptitle(f'연대별 (논문 발행 연도 기준) {title_prefix} 순위', fontsize=16, fontweight='bold')

    if len(decades) == 1:
        axes = [axes] # For consistent indexing if only one decade exists

    # 작가 분석 또는 작품 분석에 따라 이름을 가져올 컬럼 지정
    name_column = '작가_이름(원어)' if '작가_이름(원어)' in data.columns else '작품_이름(정제)'

    for i, decade in enumerate(decades):
        decade_data = data[data['분석_년대'] == decade].sort_values(by='인용_횟수', ascending=True)

        names = decade_data[name_column]
        counts = decade_data['인용_횟수']

        # 수평 막대 그래프 그리기
        axes[i].barh(names, counts, color='lightcoral' if title_prefix == '작품' else 'skyblue')
        axes[i].set_title(f'{decade} 인용 Top {len(decade_data)}', fontsize=14)
        axes[i].set_xlabel('인용 횟수', fontsize=12)
        axes[i].grid(axis='x', linestyle='--', alpha=0.7)

        # 막대 옆에 인용 횟수 표시
        for index, value in enumerate(counts):
            axes[i].text(value, index, f' {value}', va='center')

    plt.tight_layout(rect=[0, 0.03, 1, 0.97]) # Super title 공간 확보
    plt.savefig(filename, dpi=300)
    print(f"그래프 저장 완료: {filename}")


def analyze_citations():
    """
    메인 분석 함수: 작가 분석과 작품 분석을 모두 실행하고 시각화합니다.
    """
    df_filtered = load_and_preprocess_data()
    if df_filtered is None:
        return

    # 1. 작가 분석 및 CSV 저장
    author_results = analyze_authors(df_filtered)

    # 2. 작품 분석 및 CSV 저장
    work_results = analyze_works(df_filtered)

    # 3. 그래프 생성 및 저장
    if author_results is not None:
        create_and_save_plot(author_results, '작가', "author_citation_ranking_by_decade.png")

    if work_results is not None:
        create_and_save_plot(work_results, '작품', "work_citation_ranking_by_decade.png")

    print("\n[전체 분석 완료] 작가 및 작품 분석 CSV 파일이 생성되었습니다.")

# 분석 실행
analyze_citations()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 데이터 파일 경로
file_path = '/content/[현대소설연구 번역서, 원서 참고문헌 최종] (논문 발행연도 포함).csv'

# 1. CSV 파일 불러오기
try:
    df = pd.read_csv(file_path, encoding='utf-8')
except UnicodeDecodeError:
    try:
        df = pd.read_csv(file_path, encoding='cp949')
    except:
        df = pd.read_csv(file_path, encoding='euc-kr')

# 2. 데이터 전처리 및 필터링
# 'year' 칼럼을 숫자형(정수)으로 변환하고 변환 불가능한 행은 제거
df['year'] = pd.to_numeric(df['year'], errors='coerce').astype('Int64')
df.dropna(subset=['year'], inplace=True)

# 논문 발행연도 기준
df_filtered = df[(df['year'] >= 1994) & (df['year'] <= 2025)].copy()

# 3. '단행본'만 필터링
# type-name이 '단행본'인 참고문헌만 분석 대상에 포함
df_books = df_filtered[df_filtered['type-name'] == '단행본'].copy()

# 4. 연도별 인용 건수 계산
# 논문 발행연도('year')를 기준으로 그룹화하여 인용 건수를 집계
citation_counts_by_year = df_books.groupby('year').size().reset_index(name='total_book_citations')

# 시각화를 위한 폰트 설정 (한글 깨짐 방지)
plt.rcParams['font.family'] = 'NanumBarunGothic' # Windows 사용자
plt.rcParams['axes.unicode_minus'] = False # 마이너스 폰트 깨짐 방지

# 5. 결과 시각화
plt.figure(figsize=(14, 7))
sns.lineplot(
    data=citation_counts_by_year,
    x='year',
    y='total_book_citations',
    marker='o', # 데이터 포인트에 마커 표시
    color='darkblue'
)

plt.title('논문 연도별 번역서 및 원서 인용 건수 변동 추이', fontsize=16)
plt.xlabel('논문 발행 연도 (Year)', fontsize=12)
plt.ylabel('인용 건수 (Total Citation Count)', fontsize=12)
plt.grid(True, linestyle='--', alpha=0.6)
plt.xticks(range(1994, 2026, 5), rotation=45) # x축 눈금 5년 단위로 표시
plt.tight_layout()
plt.show()

# 데이터프레임의 상위 5개 행을 출력하여 결과 확인
print("\n--- 연도별 총 단행본 인용 건수 상위 5개 ---")
print(citation_counts_by_year.head())